In [ ]:
import glob
import json
import os
import re

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Saving SnR Fixed Files as JSON

In [ ]:
def extract_imports(java_code):
    # Regular expression to find import statements
    import_statements = re.findall(r"(import .*?;)", java_code)

    return import_statements

In [ ]:
snr_fixed_path = "/home/azmain/snr_fixed/"
os.chdir(snr_fixed_path)

# snr_fixed_files = [f for f in os.listdir(snr_fixed_path) if f.endswith(".java")]
snr_fixed_files = []

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

for file in sorted(glob.glob("*.java"), key=numericalSort):
    snr_fixed_files.append(file)

print(snr_fixed_files)
print()
snr_fixed_imports = []
for filename in snr_fixed_files:
    with open(os.path.join(snr_fixed_path, filename), 'r') as file:
        content = file.read()
        snr_fixed_imports.append(extract_imports(content))
print(snr_fixed_imports)

In [ ]:
def get_correct_imports(outputFiles):
    correct_outputs = []
    for output in outputFiles:
        correct_output_list = json.load(open(output))['total_imports']
        correct_output_list = ["import "+i+";" for i in correct_output_list]
        correct_outputs.append(correct_output_list)

    for import_lines in correct_outputs:
        if "import gen.R;" in import_lines:
            import_lines.remove("import gen.R;")

    return correct_outputs

In [ ]:
os.chdir("/home/azmain/snr_all_json/")

outputFiles = []

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

for file in sorted(glob.glob("*.benchmark_log.json"), key=numericalSort):
    outputFiles.append(file)

print(outputFiles)
print()

correct_imports = get_correct_imports(outputFiles)
print(correct_imports)

In [ ]:
def store_result(save_directory, code_name, expected_list, predicted_list):

    # Validate lengths
    if len(expected_list) != len(predicted_list):
        raise ValueError("The length of expected_list and predicted_list lists must be equal")

    for index, (expected, predicted) in enumerate(zip(expected_list, predicted_list)):
        file_name = ""
        if code_name == "android":
            if index<9:
                file_name = f"Android0{index+1}"
            else:
                file_name = f"Android{index+1}"
        elif code_name == "jdk":
            file_name = f"Class_{index+1}" 
        elif code_name == "hibernate":
            file_name = f"hibernate_class_{index+1}"
        elif code_name == "jodatime":
            if index<9:
                file_name = f"JodaTime0{index+1}"
            else:
                file_name = f"JodaTime{index+1}"
        elif code_name == "gwt":
            file_name = f"gwt_class_{index+1}"
        elif code_name == "xstream":
            file_name = f"xstream_class_{index+1}"

        full_path = os.path.join(save_directory, file_name)
        data = {
            "filename": file_name,
            "expected_output": expected,
            "snr_fixed_output": predicted
        }
        
        with open(f"{full_path}.json", "w") as file:
            json.dump(data, file, indent=4)

    print("JSON files generated successfully!")

In [ ]:
save_directory = "/home/azmain/snr_fixed_json/"
code_name = "android"
expected_imports = correct_imports[:50]
android_y_true = expected_imports
predicted_imports = snr_fixed_imports[:50]
android_y_pred = predicted_imports
store_result(save_directory, code_name, expected_imports, predicted_imports)

In [ ]:
save_directory = "/home/azmain/snr_fixed_json/"
code_name = "jdk"
expected_imports = correct_imports[50:73]
jdk_y_true = expected_imports
predicted_imports = snr_fixed_imports[50:73]
jdk_y_pred = predicted_imports
store_result(save_directory, code_name, expected_imports, predicted_imports)

In [ ]:
save_directory = "/home/azmain/snr_fixed_json/"
code_name = "hibernate"
expected_imports = correct_imports[173:223]
hibernate_y_true = expected_imports
predicted_imports = snr_fixed_imports[173:223]
hibernate_y_pred = predicted_imports
store_result(save_directory, code_name, expected_imports, predicted_imports)

In [ ]:
save_directory = "/home/azmain/snr_fixed_json/"
code_name = "jodatime"
expected_imports = correct_imports[73:123]
jodatime_y_true = expected_imports
predicted_imports = snr_fixed_imports[73:123]
jodatime_y_pred = predicted_imports
store_result(save_directory, code_name, expected_imports, predicted_imports)

In [ ]:
save_directory = "/home/azmain/snr_fixed_json/"
code_name = "gwt"
expected_imports = correct_imports[123:173]
gwt_y_true = expected_imports
predicted_imports = snr_fixed_imports[123:173]
gwt_y_pred = predicted_imports
store_result(save_directory, code_name, expected_imports, predicted_imports)

In [ ]:
save_directory = "/home/azmain/snr_fixed_json/"
code_name = "xstream"
expected_imports = correct_imports[223:267]
xstream_y_true = expected_imports
predicted_imports = snr_fixed_imports[223:267]
xstream_y_pred = predicted_imports
store_result(save_directory, code_name, expected_imports, predicted_imports)

# Calculating SnR Benchmark

In [ ]:
def pred_process(y_pred, y_true):
    y_pred_processed = []
    y_true_processed = []
    
    for pred, correct_imports in zip(y_pred, y_true):
        max_length = max(len(pred), len(correct_imports))
        correct_preds = list(set(pred).intersection(correct_imports))
        
        for i in range(0, max_length):
            if i<len(correct_preds):
                y_pred_processed.append(1)
                y_true_processed.append(1)
            else:
                if i<len(correct_imports):
                    y_pred_processed.append(0)
                    y_true_processed.append(1)
                else:
                    y_pred_processed.append(1)
                    y_true_processed.append(0)
            
    print(y_pred_processed)
    print()
    print(y_true_processed)
    print()
    return y_pred_processed, y_true_processed

In [ ]:
def eval_performance(y_pred, y_true):   
    print(json.dumps({
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred)
    }, indent=2))

In [ ]:
# Prediction for Android Classes

print("\nPrediction for Android Classes:\n")

y_pred_processed, y_true_processed = pred_process(android_y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for JDK Classes

print("\nPrediction for JDK Classes:\n")

y_pred_processed, y_true_processed = pred_process(jdk_y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for Hibernate Classes

print("\nPrediction for Hibernate Classes:\n")

y_pred_processed, y_true_processed = pred_process(hibernate_y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for Joda-Time Classes

print("\nPrediction for Joda-Time Classes:\n")

y_pred_processed, y_true_processed = pred_process(jodatime_y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for GWT Classes

print("\nPrediction for GWT Classes:\n")

y_pred_processed, y_true_processed = pred_process(gwt_y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction for XStream Classes

print("\nPrediction for XStream Classes:\n")

y_pred_processed, y_true_processed = pred_process(xstream_y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)